# 9.8 束搜索
- **目录**
  - 9.8.1 两种常用搜索算法：贪心搜索与穷举搜索
  - 9.8.2 束搜索基本原理


- 在9.7节中，我们逐个预测输出序列，
直到预测序列中出现特定的序列结束词元“&lt;eos&gt;”。
- 在本节中，我们将首先介绍**贪心搜索（greedy search）** 策略，
并探讨其存在的问题，然后对比其他替代策略：
**穷举搜索（exhaustive search）** 和**束搜索（beam search）** 。
- 定义搜索问题的数学符号：
  - 在任意时间步$t'$，解码器输出$y_{t'}$的概率取决于
时间步$t'$之前的输出子序列$y_1, \ldots, y_{t'-1}$
和对输入序列的信息进行编码得到的上下文变量$\mathbf{c}$。
  - 为了量化计算代价，用$\mathcal{Y}$表示输出词表，其中包含“&lt;eos&gt;”，所以这个词汇集合的基数$\left|\mathcal{Y}\right|$就是词表的大小。
  - 将输出序列的最大词元数指定为$T'$。
  - 目标是从所有$\mathcal{O}(\left|\mathcal{Y}\right|^{T'})$个
可能的输出序列中寻找理想的输出。
  - 对于所有输出序列，在“&lt;eos&gt;”之后的部分（非本句）
将在实际输出中丢弃。

- 参考书籍

<img src='../img/9_8_1.png' width=300px>

## 9.8.1 两种常用搜索算法：贪心搜索与穷举搜索
首先，让我们看看一个简单的策略：**贪心搜索**，
该策略已用于 9.7节的序列预测。
对于输出序列的每一时间步$t'$，
我们都将基于贪心搜索从$\mathcal{Y}$中找到具有最高条件概率的词元，即：

$$y_{t'} = \operatorname*{argmax}_{y \in \mathcal{Y}} P(y \mid y_1, \ldots, y_{t'-1}, \mathbf{c}) \tag{9.8.1}$$

一旦输出序列包含了“&lt;eos&gt;”或者达到其最大长度$T'$，则输出完成。

<center> <img src ='../img/s2s-prob1.svg'/></center>
<center>图9.8.1 在每个时间步，贪心搜索选择具有最高条件概率的词元</center></br>

如图9.8.1中，假设输出中有四个词元“A”、“B”、“C”和“&lt;eos&gt;”。
每个时间步下的四个数字分别表示在该时间步
生成“A”、“B”、“C”和“&lt;eos&gt;”的条件概率。
在每个时间步，贪心搜索选择具有最高条件概率的词元。
因此，将在图9.8.1中
预测输出序列“A”、“B”、“C”和“&lt;eos&gt;”。
这个输出序列的条件概率是
$0.5\times0.4\times0.4\times0.6 = 0.048$。

那么贪心搜索存在的问题是什么呢？
现实中，**最优序列（optimal sequence）** 应该是最大化
$\prod_{t'=1}^{T'} P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \mathbf{c})$
值的输出序列，这是基于输入序列生成输出序列的条件概率。
然而，贪心搜索无法保证得到最优序列。

<center><img src='../img/s2s-prob2.svg'/></center>
<center>图9.8.2 在时间步2，选择具有第二高条件概率的词元“C”（而非最高条件概率的词元）</center>

 图9.8.2中的另一个例子阐述了这个问题。
与图9.8.1不同，在时间步$2$中，
我们选择图9.8.2中的词元“C”，
它具有**第二**高的条件概率。
由于时间步$3$所基于的时间步$1$和$2$处的输出子序列已从
图9.8.1中的“A”和“B”改变为
图9.8.2中的“A”和“C”，
因此时间步$3$处的每个词元的条件概率也在图9.8.2中改变。
假设我们在时间步$3$选择词元“B”，
于是当前的时间步$4$基于前三个时间步的输出子序列“A”、“C”和“B”为条件，
这与图9.8.1中的“A”、“B”和“C”不同。
因此，在图9.8.2中的时间步$4$生成
每个词元的条件概率也不同于图9.8.1中的条件概率。
结果， 图9.8.2中的输出序列
“A”、“C”、“B”和“&lt;eos&gt;”的条件概率为
$0.5\times0.3 \times0.6\times0.6=0.054$，
这大于图9.8.1中的贪心搜索的条件概率。
这个例子说明：贪心搜索获得的输出序列
“A”、“B”、“C”和“&lt;eos&gt;”
不一定是最佳序列。

- **要点：**
  -  **贪心搜索**：在输出序列的每一时间步$t'$，我们都将基于贪心搜索从$\mathcal{Y}$中找到具有最高条件概率的词元，即$y_{t'} = \operatorname*{argmax}_{y \in \mathcal{Y}} P(y \mid y_1, \ldots, y_{t'-1}, \mathbf{c})$。
  - **结束条件**：一旦输出序列包含了“&lt;eos&gt;”或者达到其最大长度$T'$，则输出完成。
  - **贪心搜索示例**：在每个时间步，贪心搜索选择具有最高条件概率的词元。比如在一个输出序列“A”、“B”、“C”和“&lt;eos&gt;”，每个时间步下的四个数字分别表示在该时间步生成“A”、“B”、“C”和“&lt;eos&gt;”的条件概率。
  - **贪心搜索的问题**：贪心搜索无法保证得到最优序列。最优序列应该是最大化$\prod_{t'=1}^{T'} P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \mathbf{c})$值的输出序列，这是基于输入序列生成输出序列的条件概率。
  - **不是最优的例子**：在某些情况下，选择第二高的条件概率的词元也可能导致最终的序列的条件概率大于贪心搜索的序列的条件概率。

另一种常有搜索算法是穷举搜索。如果目标是获得最优序列，
我们可以考虑使用**穷举搜索（exhaustive search）**：
穷举地列举所有可能的输出序列及其条件概率，
然后计算输出条件概率最高的一个。

虽然我们可以使用穷举搜索来获得最优序列，
但其计算量$\mathcal{O}(\left|\mathcal{Y}\right|^{T'})$可能高的惊人。
例如，当$|\mathcal{Y}|=10000$和$T'=10$时，
我们需要评估$10000^{10} = 10^{40}$序列，
这是一个极大的数，现有的计算机几乎不可能计算它。
然而，贪心搜索的计算量
$\mathcal{O}(\left|\mathcal{Y}\right|T')$
要显著地小于穷举搜索。
例如，当$|\mathcal{Y}|=10000$和$T'=10$时，
我们只需要评估$10000\times10=10^5$个序列。



## 9.8.2 束搜索基本原理

那么该选取哪种序列搜索策略呢？
如果精度最重要，则显然是穷举搜索。
如果计算成本最重要，则显然是贪心搜索。
而束搜索的实际应用则**介于这两个极端之间**。

**束搜索（beam search）** 是贪心搜索的一个改进版本。
它有一个超参数，名为<b>束宽（beam size）$k$ </b>。
在时间步$1$，我们选择具有最高条件概率的$k$个词元。
这$k$个词元将分别是$k$个候选输出序列的第一个词元。
在随后的每个时间步，基于上一时间步的$k$个候选输出序列，
我们将继续从$k\left|\mathcal{Y}\right|$个可能的选择中
挑出具有最高条件概率的$k$个候选输出序列。

<center><img src='../img/beam-search.svg'/></center>
<center> 图 9.8.3束搜索过程（束宽：2，输出序列的最大长度：3）。候选输出序列是$A$、$C$、$AB$、$CE$、$ABD$和$CED$</center><br>


图9.8.3演示了束搜索的过程。
假设输出的词表只包含五个元素：
$\mathcal{Y} = \{A, B, C, D, E\}$，
其中有一个是“&lt;eos&gt;”。
设置束宽为$2$，输出序列的最大长度为$3$。
在时间步$1$，假设具有最高条件概率
$P(y_1 \mid \mathbf{c})$的词元是$A$和$C$。
在时间步$2$，我们计算所有$y_2 \in \mathcal{Y}$为：

$$\begin{aligned}P(A, y_2 \mid \mathbf{c}) = P(A \mid \mathbf{c})P(y_2 \mid A, \mathbf{c}),\\ P(C, y_2 \mid \mathbf{c}) = P(C \mid \mathbf{c})P(y_2 \mid C, \mathbf{c}),\end{aligned}  \tag{9.8.2}$$  

从这十个值中选择最大的两个，
比如$P(A, B \mid \mathbf{c})$和$P(C, E \mid \mathbf{c})$。
然后在时间步$3$，我们计算所有$y_3 \in \mathcal{Y}$为：

$$\begin{aligned}P(A, B, y_3 \mid \mathbf{c}) = P(A, B \mid \mathbf{c})P(y_3 \mid A, B, \mathbf{c}),\\P(C, E, y_3 \mid \mathbf{c}) = P(C, E \mid \mathbf{c})P(y_3 \mid C, E, \mathbf{c}),\end{aligned}  \tag{9.8.3}$$ 

从这十个值中选择最大的两个，
即$P(A, B, D \mid \mathbf{c})$和$P(C, E, D \mid  \mathbf{c})$，
我们会得到六个候选输出序列：
（1）$A$；（2）$C$；（3）$A,B$；（4）$C,E$；（5）$A,B,D$；（6）$C,E,D$。

最后，基于这六个序列（例如，丢弃包括“&lt;eos&gt;”和之后的部分），
我们获得最终候选输出序列集合。
然后我们选择其中条件概率乘积最高的序列作为输出序列：

$$ \frac{1}{L^\alpha} \log P(y_1, \ldots, y_{L}\mid \mathbf{c}) = \frac{1}{L^\alpha} \sum_{t'=1}^L \log P(y_{t'} \mid y_1, \ldots, y_{t'-1}, \mathbf{c}), \tag{9.8.4}$$


其中$L$是最终候选序列的长度，
$\alpha$通常设置为$0.75$。
因为一个较长的序列在公式9.8.4
的求和中会有更多的对数项，
因此分母中的$L^\alpha$用于惩罚长序列。

束搜索的计算量为$\mathcal{O}(k\left|\mathcal{Y}\right|T')$，
这个结果介于贪心搜索和穷举搜索之间。
实际上，贪心搜索可以看作是一种束宽为$1$的特殊类型的束搜索。
通过灵活地选择束宽，束搜索可以在正确率和计算代价之间进行权衡。

- **要点：**
  - **束搜索介绍**：束搜索（beam search）是贪心搜索的一个改进版本，其实际应用介于穷举搜索和贪心搜索之间。
  - **束宽**：束搜索有一个超参数，称为束宽（beam size）$k$。
    - 在时间步$1$，选择具有最高条件概率的$k$个词元。
    - 在后续的每个时间步，基于上一时间步的$k$个候选输出序列，继续从$k\left|\mathcal{Y}\right|$个可能的选择中挑出具有最高条件概率的$k$个候选输出序列。
  - **束搜索过程**：在每个时间步，都会计算所有可能的候选序列并从中选择条件概率最大的$k$个。
  - **最终候选输出序列**：在所有时间步完成后，我们获得最终候选输出序列集合。然后我们选择其中条件概率乘积最高的序列作为输出序列。
  - **长度惩罚**：因为一个较长的序列在公式9.8.4的求和中会有更多的对数项，因此分母中的$L^\alpha$用于惩罚长序列。
  - **计算量**：束搜索的计算量为$\mathcal{O}(k\left|\mathcal{Y}\right|T')$，这个结果介于贪心搜索和穷举搜索之间。实际上，贪心搜索可以看作是一种束宽为$1$的特殊类型的束搜索。通过灵活地选择束宽，束搜索可以在正确率和计算代价之间进行权衡。

## 小结

* 序列搜索策略包括贪心搜索、穷举搜索和束搜索。
* 贪心搜索所选取序列的计算量最小，但精度相对较低。
* 穷举搜索所选取序列的精度最高，但计算量最大。
* 束搜索通过灵活选择束宽，在正确率和计算代价之间进行权衡。

-----
- **附录：束搜索实例代码**

In [13]:
import math
from collections import defaultdict

class BeamSearch:
    def __init__(self, beam_width=2, max_length=3, alpha=0.75):
        self.beam_width = beam_width
        self.max_length = max_length
        self.alpha = alpha
        # 定义词表
        self.vocab = ['A', 'B', 'C', 'D', 'E']
        
    def get_conditional_prob(self, prev_tokens=None):
        """模拟获取条件概率的函数"""
        # 这里使用一个简单的概率分布作为示例
        probs = defaultdict(float)
        if prev_tokens is None or len(prev_tokens) == 0:  # 第一步
            probs = {'A': 0.6, 'B': 0.1, 'C': 0.2, 'D': 0.05, 'E': 0.05}
        elif prev_tokens == ('A',):  # A后面的概率
            probs = {'A': 0.1, 'B': 0.5, 'C': 0.1, 'D': 0.2, 'E': 0.1}
        elif prev_tokens == ('C',):  # C后面的概率
            probs = {'A': 0.1, 'B': 0.2, 'C': 0.1, 'D': 0.1, 'E': 0.5}
        elif prev_tokens == ('A', 'B'):  # AB后面的概率
            probs = {'A': 0.1, 'B': 0.1, 'C': 0.1, 'D': 0.6, 'E': 0.1}
        elif prev_tokens == ('C', 'E'):  # CE后面的概率
            probs = {'A': 0.1, 'B': 0.1, 'C': 0.1, 'D': 0.5, 'E': 0.2}
        else:  # 默认概率分布
            probs = {token: 0.2 for token in self.vocab}
        return probs

    def search(self):
        # 初始化候选序列
        candidates = [((), 0.0)]  # (序列, log概率)
        final_candidates = []

        # 对每个时间步进行搜索
        for step in range(self.max_length):
            print(f"\n时间步 {step + 1}:")
            all_next_candidates = []
            
            # 对当前的每个候选序列进行扩展
            for seq, score in candidates:
                probs = self.get_conditional_prob(seq)
                
                # 计算所有可能的下一个词元
                for next_token, prob in probs.items():
                    new_seq = seq + (next_token,)
                    new_score = score + math.log(prob)
                    all_next_candidates.append((new_seq, new_score))
                    print(f"候选序列: {new_seq}, log概率: {new_score:.4f}")

            # 选择前beam_width个最佳候选
            candidates = sorted(all_next_candidates, 
                             key=lambda x: x[1], 
                             reverse=True)[:self.beam_width]
            print(f"\n保留的top {self.beam_width} 候选:")
            for seq, score in candidates:
                print(f"序列: {seq}, log概率: {score:.4f}")
                final_candidates.append((seq, score))

        # 计算归一化分数
        print("\n归一化分数计算:")
        final_scores = []
        for seq, score in final_candidates:
            length = len(seq)
            if length > 0:
                normalized_score = score / (length ** self.alpha)
                final_scores.append((seq, normalized_score, score))
                print(f"序列: {seq}, 原始分数: {score:.4f}, "
                      f"长度: {length}, 归一化分数: {normalized_score:.4f}")
        
        if not final_scores:
            return None, None, None
        
        # 选择得分最高的序列
        best = []
        for score in final_scores:
            if len(score[0])==self.max_length:
                best.append(score)
        best_seq, best_norm_score, best_raw_score = max(best, key=lambda x: x[0])
        
        # 打印所有候选序列的最终得分
        print("\n所有候选序列的最终得分:")
        for seq, norm_score, raw_score in sorted(final_scores, 
                                               key=lambda x: x[1], 
                                               reverse=True):
            print(f"序列: {seq}, 归一化得分: {norm_score:.4f}, "
                  f"原始log概率: {raw_score:.4f}")
        
        return best_seq, best_norm_score, best_raw_score

# 运行束搜索
print("开始束搜索 (beam_width=2, max_length=3):")
print("----------------------------------------")
beam_search = BeamSearch(beam_width=2, max_length=3)
best_sequence, best_norm_score, best_raw_score = beam_search.search()

print("\n最终结果:")
print("----------------------------------------")
if best_sequence is not None:
    print(f"最佳序列: {best_sequence}")
    print(f"归一化得分: {best_norm_score:.4f}")
    print(f"原始log概率: {best_raw_score:.4f}")
else:
    print("未找到有效序列")

开始束搜索 (beam_width=2, max_length=3):
----------------------------------------

时间步 1:
候选序列: ('A',), log概率: -0.5108
候选序列: ('B',), log概率: -2.3026
候选序列: ('C',), log概率: -1.6094
候选序列: ('D',), log概率: -2.9957
候选序列: ('E',), log概率: -2.9957

保留的top 2 候选:
序列: ('A',), log概率: -0.5108
序列: ('C',), log概率: -1.6094

时间步 2:
候选序列: ('A', 'A'), log概率: -2.8134
候选序列: ('A', 'B'), log概率: -1.2040
候选序列: ('A', 'C'), log概率: -2.8134
候选序列: ('A', 'D'), log概率: -2.1203
候选序列: ('A', 'E'), log概率: -2.8134
候选序列: ('C', 'A'), log概率: -3.9120
候选序列: ('C', 'B'), log概率: -3.2189
候选序列: ('C', 'C'), log概率: -3.9120
候选序列: ('C', 'D'), log概率: -3.9120
候选序列: ('C', 'E'), log概率: -2.3026

保留的top 2 候选:
序列: ('A', 'B'), log概率: -1.2040
序列: ('A', 'D'), log概率: -2.1203

时间步 3:
候选序列: ('A', 'B', 'A'), log概率: -3.5066
候选序列: ('A', 'B', 'B'), log概率: -3.5066
候选序列: ('A', 'B', 'C'), log概率: -3.5066
候选序列: ('A', 'B', 'D'), log概率: -1.7148
候选序列: ('A', 'B', 'E'), log概率: -3.5066
候选序列: ('A', 'D', 'A'), log概率: -3.7297
候选序列: ('A', 'D', 'B'), log概率: -3.7297
候选序列: ('A', 'D